# Functional Design Patterns
In functional languages, most design patterns are surprisingly around functions.
Although the obvious way was to show examples in functional notation, I chose object (dot) notation and used classes in Python for the extra challenge.

These examples were enspired by OSlash library, Douglas Crockford, Computerphile and MPJ's wonderful youtube channel "Fun Fun Functions"

In [4]:
class Div:
    def __init__(self, value1, value2):
        self.value1 = value1
        self.value2 = value2
    def evaluate(self):
        iftype(self.value1) == Div:
            e1 = self.value1.evaluate()
        else:
            e1 = self.value1
        if type(self.value2) == Div:
            e2 = self.value2.evaluate()
        else:
            e2 = self.value2
        return e1//e2

In [14]:
print(Div(10,Div(4,Div(2,1))).evaluate())

5


In [15]:
print(Div(10,Div(4,Div(2,0))).evaluate())

ZeroDivisionError: integer division or modulo by zero

In [22]:
class SafeDiv:
    def __init__(self, value1, value2):
        self.value1 = value1
        self.value2 = value2
    def evaluate(self):
        if type(self.value1) == SafeDiv :
            try:
                e1 = self.value1.evaluate()
            except:
                print("Left operand evaluation failed %s" % self.value1)
                return None
        else:
            e1 = self.value1
        if(type(self.value2) == SafeDiv ):
            try:
                e2 = self.value2.evaluate()
            except:
                print("Right operand evaluation failed %s" % self.value2)
                return None
        else:
            e2 = self.value2
        if e1 == None or e2 == None:
            return None
        try:
            return e1//e2
        except:
            return None
    def __str__(self):
        return "SafeDiv(%s, %s)" % (self.value1, self.value2)

In [25]:
print(SafeDiv(10,SafeDiv(SafeDiv(2,1), 0)).evaluate())

Right operand evaluation failed SafeDiv(SafeDiv(2, 1), 0)
None


So much clutter - just because we wanted a safe version for division

# At first there were values...

In [43]:
x = 3
y = 3
print(x == y)

True


values are boring, so sometimes we want to have box.

In [44]:
class Box:
    def __init__(self, v):
        self.value = v
    def __str__(self):
        print("Box(%s)" % self.value)
x = Box(3)
y = Box(3)
print(x == y)

False


Boxes are contexts, they encapsulate the values.

# Lets talk about Arrays...

In [45]:
class Array:
    def __init__(self, *args):
        self.values = [*args]
    def map(self, func):
        return Array(*list(map(func, self.values)))
    def __str__(self):
        return self.values.__str__()

arr = Array(1,2,3)
print(arr)
arr2 = arr.map(lambda v: v*2)
print(arr2)

[1, 2, 3]
[2, 4, 6]


I've implemented a strange wrap on Python list type so the map method would be a method, instead of a global function.
The difference between calling map(lambda, l) and arr.map(lambda) is the notation - first is functional and the other is methodical.

This is just to be consice with the other examples.

As you can see, Arrays simply compose values, and also have a map method.
the map method receives a callback function, which will get one value at a time.

Thus Arrays are Functors.

# Functors - Objects with a map function

Functors are boxes with a "map" method. The idea behind them is that they encapsulate the value, and allows you via the map function access to those values, manipulate them and pack them in a Functor.

In [46]:
class Functor:
    def __init__(self, value):
        self.value = value
    def map(self, func):
        return Functor(func(self.value))
    def __str__(self):
        return "Functor(%s)" % self.value

In [47]:
Five = Functor(5)
print(Five)
Ten = Five.map(lambda v: v*2)
print(Ten)

Functor(5)
Functor(10)


# Functors are very useful, we use them daily. But can we do better?

For instance, what will happen if our callback function will return a Functor?

In [48]:
BadTen = Five.map(lambda v: Functor(v*2))
print(BadTen)

Functor(Functor(10))


This makes me :(

# Monad - Objects with flat_map function

Lets see the following code: It uses dog.ceo API to get random pictures of dogs. I've decided to use asyncio Futures in Python. 

In [49]:
import asyncio
import requests
from IPython.display import Image 

def get_url(url):
    print("requesting %s" % url)
    return requests.get(url)

def fetch_image(url):
    print("generating image %s" % url)
    return Image(url=url)

def random_dog_callback(result_future):
    url = result_future.result().json()["message"]
    fututre_image = loop.run_in_executor(None, fetch_image, url)
    fututre_image.add_done_callback(lambda img: display(img.result()))

loop = asyncio.get_running_loop()
future_result = loop.run_in_executor(None, get_url, "https://dog.ceo/api/breeds/image/random")
print("waiting for random dog...")
future_result.add_done_callback(random_dog_callback)


waiting for random dog...requesting https://dog.ceo/api/breeds/image/random

generating image https://images.dog.ceo/breeds/cattledog-australian/IMG_5481.jpg


## The Future doesn't look bright
Futures in Python don't have a very good API, and working with asyncio Event Loop in python is no fun either. This code is not reusable, and has a lot of clutter just to get an image asyncronously. 

If we need to do it many times in our code base, in this constilation we will have to pad our code with get_running_loops and run_in_executor, and we also have a potential for callback hells.

We can write a Functor that will encapsulate the clutter, but our callbacks can be asyncronous as well. But, we can use the Monad Design Pattern.

A monad is an object that has a flat_map method, which is very simillar to map, but can also handle monads as values.



## I promise you things will get better now...

In Javascript, there are promises, lets implement something simillar in Python. It wraps the Future objects but simplifies its API while also hiding the over head of the asyncio Event Loop API.

In [50]:
class Vow:
    def __init__(self, value=None):
        self.value = value
    def maybe_wrap(value):
        if type(value) == Vow:
            return value
        return Vow(value)
        
    def flat_map(self, func):
        loop = asyncio.get_running_loop()
        # If value is a Vow, simply chain the func
        if type(self.value) == Vow:
            return Vow(loop.run_in_executor(None, lambda: self.value.flat_map(func)))
        # If value is a Future value, then once it will be done execute func on the value
        if asyncio.isfuture(self.value):
            new_future = loop.create_future()
            self.value.add_done_callback(lambda _: new_future.set_result(Vow.maybe_wrap(self.value.result()).flat_map(func)))
            return Vow(new_future)
        # If it is a simple value then tell the event loop to execute it later
        return Vow(loop.run_in_executor(None, lambda: func(self.value)))

    
    def when(self, func):
        return self.flat_map(func)

In [51]:
Vow("https://dog.ceo/api/breeds/image/random") \
    .flat_map(get_url) \
    .flat_map(lambda response: Vow(response.json())) \
    .flat_map(lambda json: json["message"]) \
    .flat_map(fetch_image) \
    .flat_map(display)

print("")

requesting https://dog.ceo/api/breeds/image/random

generating image https://images.dog.ceo/breeds/malinois/n02105162_5327.jpg


### And now with a better method name :)

In [52]:
Vow("https://dog.ceo/api/breeds/image/random") \
    .when(get_url)\
    .when(lambda response: Vow(response.json())) \
    .when(lambda json: json["message"]) \
    .when(fetch_image) \
    .when(display)
print("")

requesting https://dog.ceo/api/breeds/image/random

generating image https://images.dog.ceo/breeds/retriever-flatcoated/n02099267_4098.jpg


In [53]:
def whened(func):
    def whened_func(self):
        result = self.when(lambda v: func(self, v))
        return ImageFetcher(result.value)
    return whened_func

class ImageFetcher(Vow):
    @whened
    def get_url(self, url):
        print("requesting %s" % url)
        return requests.get(url)
    
    @whened
    def get_image_url(self, response):
        json = response.json()
        return json["message"]

    @whened
    def fetch_image(self, url):
        print("generating image %s" % url)
        return Image(url=url)
    
    @whened
    def display(self, img):
        display(img)
        


In [54]:
ImageFetcher("https://dog.ceo/api/breeds/image/random").get_url().get_image_url().fetch_image().display()
ImageFetcher("https://dog.ceo/api/breeds/image/random").get_url().get_image_url().fetch_image().display()
ImageFetcher("https://dog.ceo/api/breeds/image/random").get_url().get_image_url().fetch_image().display()
print("")


requesting https://dog.ceo/api/breeds/image/randomrequesting https://dog.ceo/api/breeds/image/random
requesting https://dog.ceo/api/breeds/image/random

generating image https://images.dog.ceo/breeds/setter-gordon/n02101006_3868.jpg


generating image https://images.dog.ceo/breeds/whippet/n02091134_17394.jpg


generating image https://images.dog.ceo/breeds/retriever-curly/n02099429_1412.jpg
